# 💬 Client - LLM 기본 사용법

이 노트북은 beanllm의 핵심 기능인 `Client`를 사용하여 다양한 LLM Provider와 상호작용하는 방법을 학습합니다.

## 📋 목차

- [1. Client 기본 사용](#1-client-기본-사용)
- [2. Provider 전환](#2-provider-전환)
- [3. 스트리밍 응답](#3-스트리밍-응답)
- [4. 파라미터 조정](#4-파라미터-조정)
- [5. Model Registry 활용](#5-model-registry-활용)
- [6. 고급 기능](#6-고급-기능)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ Client를 사용하여 LLM과 대화할 수 있습니다
- ✅ 다양한 Provider 간 전환을 할 수 있습니다
- ✅ 스트리밍 응답을 처리할 수 있습니다
- ✅ 파라미터를 조정하여 응답을 제어할 수 있습니다
- ✅ Model Registry를 활용하여 모델을 선택할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- 최소 하나의 LLM Provider API 키 설정


## 1. Client 기본 사용

`Client`는 beanllm의 핵심 인터페이스로, 다양한 LLM Provider를 통일된 방식으로 사용할 수 있게 해줍니다. Provider에 상관없이 동일한 API를 사용할 수 있어, 코드 변경 없이 다른 Provider로 전환할 수 있습니다.

### 1.1 Client 생성

`Client`를 생성하는 가장 간단한 방법은 모델 이름만 지정하는 것입니다. beanllm은 자동으로 적절한 Provider를 선택하고 연결합니다.

> 💡 **핵심 개념**: 
> - Client는 Provider에 상관없이 동일한 API를 제공합니다
> - 내부적으로 `ParameterAdapter`가 자동으로 파라미터를 변환합니다
> - Model Registry를 통해 사용 가능한 모델을 자동으로 인식합니다
> - Provider별 차이점을 자동으로 처리합니다

> ⚠️ **주의사항**: 
> - API 키가 환경 변수에 설정되어 있어야 합니다
> - 사용하려는 Provider의 SDK가 설치되어 있어야 합니다


In [ ]:
import asyncio
from beanllm import Client, get_model_registry

# ============================================
# Client 생성 (기본 방법)
# ============================================

print("=" * 60)
print("Client 생성 - 기본 방법")
print("=" * 60)

# 방법 1: 모델 이름만 지정 (가장 간단)
print("\n[방법 1] 모델 이름만 지정:")
client1 = Client(model="gpt-4o")
print(f"  ✅ Client 생성 완료!")
print(f"  📊 모델: {client1.model}")
print(f"  📊 Provider: {getattr(client1, 'provider', '자동 선택')}")

# 방법 2: Provider 명시적 지정
print("\n[방법 2] Provider 명시적 지정:")
# client2 = Client(model="gpt-4o", provider="openai")
# print(f"  ✅ Client 생성 완료!")
# print(f"  📊 모델: {client2.model}")
# print(f"  📊 Provider: {client2.provider}")

# 방법 3: 추가 옵션 지정
print("\n[방법 3] 추가 옵션 지정:")
# client3 = Client(
#     model="gpt-4o",
#     temperature=0.7,
#     max_tokens=1000,
#     timeout=30
# )
# print(f"  ✅ Client 생성 완료!")
# print(f"  📊 모델: {client3.model}")
# print(f"  📊 Temperature: {client3.temperature if hasattr(client3, 'temperature') else 'N/A'}")

print("\n💡 Client 생성 옵션:")
print("  - model: 모델 이름 (필수)")
print("  - provider: Provider 이름 (선택, 자동 선택됨)")
print("  - temperature: 창의성 조절 (0.0-2.0, 기본값: 1.0)")
print("  - max_tokens: 최대 토큰 수 (기본값: 모델별 다름)")
print("  - timeout: 타임아웃 (초, 기본값: 30)")

# Model Registry로 사용 가능한 모델 확인
print("\n" + "=" * 60)
print("사용 가능한 모델 확인")
print("=" * 60)

try:
    registry = get_model_registry()
    models = registry.get_all_models()

    print(f"\n📊 전체 모델 수: {len(models)}개")
    print("\n🔍 Provider별 모델 수:")
    by_provider = registry.get_models_by_provider()
    for provider, provider_models in by_provider.items():
        print(f"  - {provider}: {len(provider_models)}개")

    print("\n💡 인기 모델 예시:")
    popular_models = ["gpt-4o", "claude-sonnet-4-20250514", "gemini-2.5-pro"]
    for model_name in popular_models:
        model = registry.get_model(model_name)
        if model:
            print(f"  - {model.name} ({model.provider})")
except Exception as e:
    print(f"  ⚠️ Model Registry 확인 실패: {e}")

print("\n" + "=" * 60)
print("✅ Client 생성 방법 학습 완료!")
print("=" * 60)

### 1.2 기본 채팅

가장 기본적인 사용법은 `chat()` 메서드를 사용하는 것입니다. 이 메서드는 메시지 리스트를 받아 LLM의 응답을 반환합니다.

> 💡 **메시지 형식**:
> - `role`: "system", "user", "assistant" 중 하나
> - `content`: 메시지 내용 (문자열)
> - System 메시지는 모델의 행동을 정의합니다
> - User 메시지는 사용자의 질문이나 요청입니다
> - Assistant 메시지는 이전 대화의 맥락을 유지하는 데 사용됩니다


In [ ]:
# ============================================
# 기본 채팅 예제
# ============================================


async def basic_chat():
    """
    기본 채팅 예제

    가장 간단한 형태의 LLM 대화입니다.
    단일 사용자 메시지를 보내고 응답을 받습니다.
    """
    print("=" * 60)
    print("기본 채팅 예제")
    print("=" * 60)

    client = Client(model="gpt-4o")

    # 단일 메시지
    user_message = "안녕하세요! 간단히 자기소개를 해주세요."
    print(f"\n💬 사용자: {user_message}")

    # LLM 호출
    # response = await client.chat(
    #     messages=[{"role": "user", "content": user_message}]
    # )

    # 응답 출력
    # print(f"\n🤖 AI: {response.content}")

    # 메타데이터 확인
    # print(f"\n📊 응답 메타데이터:")
    # print(f"  - 모델: {response.model}")
    # if hasattr(response, 'usage'):
    #     print(f"  - 입력 토큰: {response.usage.prompt_tokens}")
    #     print(f"  - 출력 토큰: {response.usage.completion_tokens}")
    #     print(f"  - 총 토큰: {response.usage.total_tokens}")
    # if hasattr(response, 'finish_reason'):
    #     print(f"  - 완료 이유: {response.finish_reason}")

    print("\n💡 기본 채팅의 특징:")
    print("  - 가장 간단한 사용법")
    print("  - 단일 질문-답변에 적합")
    print("  - 대화 맥락 없음 (각 호출이 독립적)")

    # return response


# 실행 (주석 해제)
# response = await basic_chat()

# ============================================
# System 메시지 사용 예제
# ============================================


async def system_message_example():
    """
    System 메시지 사용 예제

    System 메시지를 사용하면 모델의 행동을 정의할 수 있습니다.
    """
    print("\n" + "=" * 60)
    print("System 메시지 사용 예제")
    print("=" * 60)

    client = Client(model="gpt-4o")

    # System 메시지로 모델의 역할 정의
    messages = [
        {
            "role": "system",
            "content": "당신은 친절하고 전문적인 Python 프로그래밍 튜터입니다. "
            "복잡한 개념을 쉽게 설명하고, 실용적인 예제를 제공합니다.",
        },
        {"role": "user", "content": "Python의 리스트 컴프리헨션에 대해 설명해주세요."},
    ]

    print("\n📝 메시지 구성:")
    for msg in messages:
        print(f"  [{msg['role'].upper()}]: {msg['content'][:50]}...")

    # LLM 호출
    # response = await client.chat(messages=messages)
    # print(f"\n🤖 AI: {response.content}")

    print("\n💡 System 메시지의 장점:")
    print("  - 모델의 행동을 일관되게 제어")
    print("  - 특정 역할이나 톤 설정")
    print("  - 응답 스타일 커스터마이징")


# 실행 (주석 해제)
# await system_message_example()

print("\n" + "=" * 60)
print("✅ 기본 채팅 학습 완료!")
print("=" * 60)

### 1.3 대화형 채팅

여러 메시지를 주고받는 대화를 구현할 수 있습니다. 대화 히스토리를 유지하면 모델이 이전 대화의 맥락을 이해하고 일관된 대화를 이어갈 수 있습니다.

> 💡 **대화 히스토리 관리**:
> - 각 응답 후 Assistant 메시지를 히스토리에 추가
> - User 메시지를 히스토리에 추가
> - 전체 히스토리를 다음 호출에 전달
> - 토큰 제한을 고려하여 히스토리 길이 관리 필요


In [ ]:
# ============================================
# 대화형 채팅 예제
# ============================================


async def conversation_example():
    """
    대화형 채팅 예제

    여러 턴의 대화를 이어가는 방법을 보여줍니다.
    대화 히스토리를 유지하여 맥락을 보존합니다.
    """
    print("=" * 60)
    print("대화형 채팅 예제")
    print("=" * 60)

    client = Client(model="gpt-4o")

    # 대화 히스토리 초기화
    messages = [
        {"role": "system", "content": "당신은 친절하고 지식이 풍부한 AI 어시스턴트입니다."},
        {"role": "user", "content": "Python이란 무엇인가요?"},
    ]

    print("\n[턴 1]")
    print(f"💬 사용자: {messages[-1]['content']}")

    # 첫 번째 응답
    # response1 = await client.chat(messages=messages)
    # print(f"🤖 AI: {response1.content}\n")

    # 대화 히스토리에 추가
    # messages.append({"role": "assistant", "content": response1.content})
    # messages.append({"role": "user", "content": "그럼 Java와 비교해주세요."})

    print("\n[턴 2]")
    print(
        f"💬 사용자: {messages[-1]['content'] if len(messages) > 2 else '그럼 Java와 비교해주세요.'}"
    )

    # 두 번째 응답
    # response2 = await client.chat(messages=messages)
    # print(f"🤖 AI: {response2.content}\n")

    # 대화 히스토리에 추가
    # messages.append({"role": "assistant", "content": response2.content})
    # messages.append({"role": "user", "content": "어떤 프로젝트에 Python을 사용하는 게 좋을까요?"})

    print("\n[턴 3]")
    print(
        f"💬 사용자: {messages[-1]['content'] if len(messages) > 4 else '어떤 프로젝트에 Python을 사용하는 게 좋을까요?'}"
    )

    # 세 번째 응답
    # response3 = await client.chat(messages=messages)
    # print(f"🤖 AI: {response3.content}\n")

    # 대화 통계
    # print("=" * 60)
    # print("대화 통계")
    # print("=" * 60)
    # print(f"  총 대화 턴: {len([m for m in messages if m['role'] == 'user'])}턴")
    # print(f"  총 메시지 수: {len(messages)}개")
    # print(f"  System 메시지: {len([m for m in messages if m['role'] == 'system'])}개")
    # print(f"  User 메시지: {len([m for m in messages if m['role'] == 'user'])}개")
    # print(f"  Assistant 메시지: {len([m for m in messages if m['role'] == 'assistant'])}개")

    print("\n💡 대화형 채팅의 특징:")
    print("  - 이전 대화의 맥락을 유지")
    print("  - 자연스러운 대화 흐름")
    print("  - 참조 가능 (예: '그것', '위에서 말한 것')")
    print("  - 토큰 사용량이 증가하므로 주의 필요")


# 실행 (주석 해제)
# await conversation_example()

# ============================================
# 대화 히스토리 관리 예제
# ============================================


async def conversation_history_management():
    """
    대화 히스토리 관리 예제

    긴 대화의 경우 히스토리를 효율적으로 관리하는 방법을 보여줍니다.
    """
    print("\n" + "=" * 60)
    print("대화 히스토리 관리 예제")
    print("=" * 60)

    client = Client(model="gpt-4o")

    # 대화 히스토리
    messages = [{"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."}]

    # 대화 시뮬레이션
    conversations = [
        "안녕하세요!",
        "오늘 날씨가 어때요?",
        "Python에 대해 알려주세요.",
        "그럼 JavaScript는요?",
    ]

    print("\n📝 대화 시뮬레이션:")
    # for i, user_msg in enumerate(conversations, 1):
    #     messages.append({"role": "user", "content": user_msg})
    #     print(f"\n[턴 {i}]")
    #     print(f"  사용자: {user_msg}")
    #
    #     # 응답 받기
    #     response = await client.chat(messages=messages)
    #     print(f"  AI: {response.content[:100]}...")
    #
    #     # 히스토리에 추가
    #     messages.append({"role": "assistant", "content": response.content})
    #
    #     # 히스토리 길이 확인
    #     total_chars = sum(len(m["content"]) for m in messages)
    #     print(f"  히스토리 길이: {len(messages)}개 메시지, {total_chars}자")

    print("\n💡 히스토리 관리 팁:")
    print("  - 토큰 제한을 고려하여 오래된 메시지 제거")
    print("  - 중요한 맥락만 유지")
    print("  - 요약을 사용하여 히스토리 압축")
    print("  - System 메시지는 항상 유지")


# 실행 (주석 해제)
# await conversation_history_management()

print("\n" + "=" * 60)
print("✅ 대화형 채팅 학습 완료!")
print("=" * 60)

## 2. Provider 전환

beanllm의 가장 강력한 기능 중 하나는 **Provider 간 자유로운 전환**입니다. 같은 코드로 다른 Provider를 사용할 수 있어, 비용, 성능, 기능에 따라 최적의 Provider를 선택할 수 있습니다.

### 2.1 Provider별 Client 생성

각 Provider는 고유한 특성을 가지고 있습니다. 예를 들어, OpenAI는 빠른 응답 속도, Anthropic은 긴 컨텍스트, Google은 비용 효율성을 제공합니다. beanllm은 이러한 차이를 자동으로 처리하므로, 코드 변경 없이 Provider를 전환할 수 있습니다.

> 💡 **Provider 선택 가이드**:
> - **OpenAI**: 빠른 응답, 높은 품질, 넓은 모델 선택
> - **Anthropic**: 긴 컨텍스트, 안전성, 복잡한 추론
> - **Google Gemini**: 비용 효율, 멀티모달, 빠른 응답
> - **DeepSeek**: 저렴한 비용, 한국어 지원
> - **Perplexity**: 웹 검색 통합, 최신 정보
> - **Ollama**: 로컬 실행, 프라이버시, 무료


In [ ]:
# ============================================
# Provider 비교 예제
# ============================================


async def provider_comparison():
    """
    Provider 비교 예제

    같은 질문에 대해 여러 Provider의 응답을 비교합니다.
    각 Provider의 특성을 확인할 수 있습니다.
    """
    print("=" * 60)
    print("Provider 비교 예제")
    print("=" * 60)

    question = "양자 컴퓨팅에 대해 간단히 설명해주세요."
    print(f"\n❓ 질문: {question}\n")

    providers_to_test = [
        {"name": "OpenAI", "model": "gpt-4o", "emoji": "🔵", "description": "빠른 응답, 높은 품질"},
        {
            "name": "Anthropic",
            "model": "claude-sonnet-4-20250514",
            "emoji": "🟣",
            "description": "긴 컨텍스트, 안전성",
        },
        {
            "name": "Google Gemini",
            "model": "gemini-2.5-pro",
            "emoji": "🟢",
            "description": "비용 효율, 멀티모달",
        },
        {
            "name": "DeepSeek",
            "model": "deepseek-chat",
            "emoji": "🟡",
            "description": "저렴한 비용, 한국어 지원",
        },
    ]

    results = []

    for provider_info in providers_to_test:
        print(
            f"\n{provider_info['emoji']} {provider_info['name']} ({provider_info['description']}):"
        )
        print("-" * 60)

        try:
            client = Client(model=provider_info["model"])
            # response = await client.chat(
            #     messages=[{"role": "user", "content": question}]
            # )
            #
            # print(f"  응답: {response.content[:200]}...")
            #
            # # 응답 시간 측정 (시뮬레이션)
            # import time
            # start_time = time.time()
            # # 실제 호출은 위에서 이미 했으므로 여기서는 시뮬레이션
            # elapsed_time = time.time() - start_time
            #
            # results.append({
            #     "provider": provider_info['name'],
            #     "model": provider_info['model'],
            #     "response_length": len(response.content),
            #     "response_preview": response.content[:100]
            # })

            print(f"  ✅ 성공 (시뮬레이션)")

        except Exception as e:
            print(f"  ❌ 오류: {e}")
            print(f"  💡 API 키가 설정되어 있는지 확인하세요.")

    # 결과 비교
    # if results:
    #     print("\n" + "=" * 60)
    #     print("Provider 비교 결과")
    #     print("=" * 60)
    #     for result in results:
    #         print(f"\n[{result['provider']}]")
    #         print(f"  모델: {result['model']}")
    #         print(f"  응답 길이: {result['response_length']}자")
    #         print(f"  응답 미리보기: {result['response_preview']}...")

    print("\n💡 Provider 선택 팁:")
    print("  - 빠른 응답이 필요: OpenAI, Google Gemini")
    print("  - 긴 컨텍스트 필요: Anthropic Claude")
    print("  - 비용 절감: DeepSeek, Google Gemini")
    print("  - 로컬 실행: Ollama")
    print("  - 최신 정보: Perplexity")


# 실행 (주석 해제)
# await provider_comparison()

# ============================================
# 동적 Provider 전환 예제
# ============================================


async def dynamic_provider_switching():
    """
    동적 Provider 전환 예제

    런타임에 Provider를 전환하여 사용할 수 있습니다.
    """
    print("\n" + "=" * 60)
    print("동적 Provider 전환 예제")
    print("=" * 60)

    question = "머신러닝의 기본 개념을 설명해주세요."

    # Provider 리스트
    providers = [
        ("gpt-4o", "OpenAI"),
        ("claude-sonnet-4-20250514", "Anthropic"),
        ("gemini-2.5-pro", "Google"),
    ]

    print(f"\n❓ 질문: {question}")
    print(f"\n🔄 여러 Provider로 시도:\n")

    # 각 Provider로 시도 (첫 번째 성공한 것 사용)
    # for model, provider_name in providers:
    #     try:
    #         print(f"[시도] {provider_name} ({model})...")
    #         client = Client(model=model)
    #         response = await client.chat(
    #             messages=[{"role": "user", "content": question}]
    #         )
    #         print(f"  ✅ 성공! {provider_name} 사용")
    #         print(f"  응답: {response.content[:150]}...")
    #         break
    #     except Exception as e:
    #         print(f"  ❌ 실패: {e}")
    #         continue
    # else:
    #     print("  ⚠️ 모든 Provider 실패")

    print("\n💡 동적 전환의 활용:")
    print("  - Fallback 메커니즘 구현")
    print("  - 비용 최적화 (저렴한 Provider 우선)")
    print("  - 성능 최적화 (빠른 Provider 우선)")
    print("  - A/B 테스트")


# 실행 (주석 해제)
# await dynamic_provider_switching()

print("\n" + "=" * 60)
print("✅ Provider 전환 학습 완료!")
print("=" * 60)

### 2.2 자동 Provider 선택

모델 이름만 지정하면 자동으로 적절한 Provider를 선택합니다.


In [ ]:
from beanllm import get_model_registry

# Model Registry로 사용 가능한 모델 확인
registry = get_model_registry()

# 사용 가능한 모델 목록
print("📊 사용 가능한 모델 (처음 10개):")
models = registry.get_all_models()
for i, model in enumerate(models[:10], 1):
    print(f"  {i}. {model.name} ({model.provider})")

# 특정 Provider의 모델
print("\n🔵 OpenAI 모델:")
openai_models = registry.get_models_by_provider().get("openai", [])
for model in openai_models[:5]:
    print(f"  - {model.name}")

## 3. 스트리밍 응답

긴 응답을 실시간으로 받으려면 `stream_chat()` 메서드를 사용합니다. 스트리밍은 응답이 생성되는 대로 청크 단위로 받아볼 수 있어, 사용자 경험을 크게 향상시킵니다.

> 💡 **스트리밍의 장점**: 
> - **사용자 경험 향상**: 즉시 응답이 시작되어 대기 시간 감소
> - **긴 응답 처리 효율성**: 전체 응답을 기다리지 않고 점진적으로 처리
> - **실시간 피드백**: 응답 생성 과정을 실시간으로 확인 가능
> - **메모리 효율성**: 전체 응답을 메모리에 저장하지 않고 스트림으로 처리

> ⚠️ **주의사항**:
> - 스트리밍은 비동기 함수이므로 `async for`를 사용해야 합니다
> - 각 청크는 문자열이므로 수집하여 전체 응답을 구성할 수 있습니다


In [ ]:
# ============================================
# 기본 스트리밍 예제
# ============================================


async def streaming_example():
    """
    기본 스트리밍 응답 예제

    스트리밍을 사용하여 응답을 실시간으로 받아봅니다.
    """
    print("=" * 60)
    print("기본 스트리밍 예제")
    print("=" * 60)

    client = Client(model="gpt-4o")

    question = "Python의 장점 5가지를 설명해주세요."
    print(f"\n❓ 질문: {question}")
    print("\n📡 스트리밍 응답 시작:\n")
    print("🤖 AI: ", end="", flush=True)

    # 스트리밍 응답 수집
    full_response = ""
    chunk_count = 0

    # async for chunk in client.stream_chat(
    #     messages=[{"role": "user", "content": question}]
    # ):
    #     print(chunk, end="", flush=True)
    #     full_response += chunk
    #     chunk_count += 1

    print(f"\n\n✅ 스트리밍 완료!")
    print(f"📊 통계:")
    print(f"  - 전체 응답 길이: {len(full_response)}자")
    print(f"  - 수신된 청크 수: {chunk_count}개")
    print(f"  - 평균 청크 크기: {len(full_response) // max(chunk_count, 1)}자")

    print("\n💡 스트리밍의 특징:")
    print("  - 응답이 생성되는 대로 즉시 표시")
    print("  - 긴 응답에서 특히 유용")
    print("  - 사용자가 응답을 기다리는 느낌 감소")


# 실행 (주석 해제)
# await streaming_example()

# ============================================
# 스트리밍 응답 처리 예제
# ============================================


async def streaming_with_processing():
    """
    스트리밍 응답 처리 예제

    스트리밍 응답을 받으면서 동시에 처리할 수 있습니다.
    """
    print("\n" + "=" * 60)
    print("스트리밍 응답 처리 예제")
    print("=" * 60)

    client = Client(model="gpt-4o")

    question = "머신러닝의 주요 알고리즘 3가지를 설명해주세요."
    print(f"\n❓ 질문: {question}")

    # 스트리밍 응답을 받으면서 처리
    sentences = []
    current_sentence = ""

    print("\n📡 스트리밍 응답 (문장 단위 처리):\n")

    # async for chunk in client.stream_chat(
    #     messages=[{"role": "user", "content": question}]
    # ):
    #     current_sentence += chunk
    #     print(chunk, end="", flush=True)
    #
    #     # 문장이 완성되면 처리
    #     if chunk in [".", "!", "?", "\n"]:
    #         if current_sentence.strip():
    #             sentences.append(current_sentence.strip())
    #             current_sentence = ""

    # 마지막 문장 처리
    # if current_sentence.strip():
    #     sentences.append(current_sentence.strip())

    print(f"\n\n✅ 처리 완료!")
    print(f"📊 추출된 문장 수: {len(sentences)}개")
    # for i, sentence in enumerate(sentences[:5], 1):
    #     print(f"  {i}. {sentence[:80]}...")

    print("\n💡 스트리밍 처리 활용:")
    print("  - 실시간 텍스트 분석")
    print("  - 문장 단위 처리")
    print("  - 키워드 추출")
    print("  - 진행 상황 표시")


# 실행 (주석 해제)
# await streaming_with_processing()

print("\n" + "=" * 60)
print("✅ 스트리밍 응답 학습 완료!")
print("=" * 60)

### 3.1 스트리밍 응답 수집

스트리밍 응답을 수집하여 나중에 사용할 수 있습니다.


In [ ]:
async def collect_streaming():
    """스트리밍 응답 수집"""
    client = Client(model="gpt-4o")

    chunks = []
    async for chunk in client.stream_chat(
        messages=[{"role": "user", "content": "간단한 시를 써주세요."}]
    ):
        chunks.append(chunk)
        print(chunk, end="", flush=True)

    full_text = "".join(chunks)
    print(f"\n\n📊 수집된 청크 수: {len(chunks)}")
    print(f"📊 전체 텍스트 길이: {len(full_text)}자")

    return full_text


# 실행 (주석 해제)
# result = await collect_streaming()
print("💡 위의 주석을 해제하여 실행하세요.")

## 4. 파라미터 조정

LLM의 응답을 제어하기 위해 다양한 파라미터를 조정할 수 있습니다.

### 4.1 Temperature (창의성)

`temperature`는 응답의 창의성을 제어합니다:
- **낮은 값 (0.0-0.5)**: 일관되고 예측 가능한 응답
- **중간 값 (0.5-1.0)**: 균형잡힌 응답
- **높은 값 (1.0-2.0)**: 창의적이고 다양한 응답


In [ ]:
async def temperature_comparison():
    """Temperature 비교"""
    question = "인공지능의 미래에 대해 한 문장으로 설명해주세요."

    temperatures = [0.1, 0.7, 1.5]

    for temp in temperatures:
        print(f"\n🌡️ Temperature: {temp}")
        client = Client(model="gpt-4o")
        response = await client.chat(
            messages=[{"role": "user", "content": question}],
            temperature=temp,
        )
        print(f"  {response.content}")


# 실행 (주석 해제)
# await temperature_comparison()
print("💡 위의 주석을 해제하여 실행하세요.")

### 4.2 Max Tokens (응답 길이)

`max_tokens`는 응답의 최대 길이를 제한합니다.

> ⚠️ **주의**: Provider마다 파라미터 이름이 다를 수 있지만, beanllm이 자동으로 변환합니다:
> - OpenAI: `max_tokens`
> - Google: `max_output_tokens`
> - Ollama: `num_predict`


In [ ]:
async def max_tokens_example():
    """Max Tokens 예제"""
    question = "Python의 특징을 설명해주세요."

    # 짧은 응답
    print("📏 짧은 응답 (max_tokens=50):")
    client = Client(model="gpt-4o")
    response = await client.chat(
        messages=[{"role": "user", "content": question}],
        max_tokens=50,
    )
    print(f"  {response.content}\n")

    # 긴 응답
    print("📏 긴 응답 (max_tokens=500):")
    response = await client.chat(
        messages=[{"role": "user", "content": question}],
        max_tokens=500,
    )
    print(f"  {response.content[:200]}...")


# 실행 (주석 해제)
# await max_tokens_example()
print("💡 위의 주석을 해제하여 실행하세요.")

### 4.3 기타 파라미터

다양한 파라미터를 조합하여 응답을 세밀하게 제어할 수 있습니다.


In [ ]:
async def advanced_parameters():
    """고급 파라미터 예제"""
    client = Client(model="gpt-4o")

    response = await client.chat(
        messages=[{"role": "user", "content": "창의적인 스토리를 써주세요."}],
        temperature=1.2,  # 창의성 높음
        max_tokens=300,  # 최대 길이
        top_p=0.9,  # Nucleus sampling
        frequency_penalty=0.5,  # 반복 억제
        presence_penalty=0.3,  # 주제 다양성
    )

    print("📝 생성된 스토리:")
    print(response.content)


# 실행 (주석 해제)
# await advanced_parameters()
print("💡 위의 주석을 해제하여 실행하세요.")

## 5. Model Registry 활용

Model Registry를 사용하여 사용 가능한 모델을 탐색하고 선택할 수 있습니다.

### 5.1 모델 정보 조회


In [ ]:
from beanllm import get_model_registry

registry = get_model_registry()

# 특정 모델 정보
model_name = "gpt-4o"
model_info = registry.get_model(model_name)

if model_info:
    print(f"📊 모델 정보: {model_name}")
    print(f"  - Provider: {model_info.provider}")
    print(f"  - Context Window: {model_info.context_window:,} tokens")
    print(f"  - Supports Streaming: {model_info.supports_streaming}")
    print(
        f"  - Max Tokens: {model_info.max_tokens if hasattr(model_info, 'max_tokens') else 'N/A'}"
    )
else:
    print(f"❌ 모델 '{model_name}'을 찾을 수 없습니다.")

### 5.2 모델 필터링

특정 조건에 맞는 모델을 찾을 수 있습니다.


In [ ]:
# Provider별 모델
print("🔵 OpenAI 모델:")
openai_models = registry.get_models_by_provider().get("openai", [])
for model in openai_models[:5]:
    print(f"  - {model.name}")

# 스트리밍 지원 모델
print("\n📡 스트리밍 지원 모델:")
all_models = registry.get_all_models()
streaming_models = [m for m in all_models if m.supports_streaming]
print(f"  총 {len(streaming_models)}개 모델이 스트리밍을 지원합니다.")

## 6. 고급 기능

### 6.1 비용 추적

토큰 사용량과 비용을 추적할 수 있습니다.


In [ ]:
from beanllm import estimate_cost, count_tokens


async def cost_tracking():
    """비용 추적 예제"""
    client = Client(model="gpt-4o")

    messages = [{"role": "user", "content": "Python에 대해 설명해주세요."}]

    # 토큰 수 계산
    input_tokens = count_tokens(str(messages), model="gpt-4o")
    print(f"📊 입력 토큰: {input_tokens}")

    # 응답 받기
    response = await client.chat(messages=messages)

    # 출력 토큰 계산
    output_tokens = count_tokens(response.content, model="gpt-4o")
    print(f"📊 출력 토큰: {output_tokens}")

    # 비용 추정
    cost = estimate_cost(
        model="gpt-4o",
        input_tokens=input_tokens,
        output_tokens=output_tokens,
    )
    print(f"💰 예상 비용: ${cost:.6f}")


# 실행 (주석 해제)
# await cost_tracking()
print("💡 위의 주석을 해제하여 실행하세요.")

### 6.2 에러 처리

실제 프로덕션 환경에서는 에러 처리가 중요합니다.


In [ ]:
async def error_handling_example():
    """에러 처리 예제"""
    client = Client(model="gpt-4o")

    try:
        response = await client.chat(
            messages=[{"role": "user", "content": "테스트"}],
            max_tokens=10,  # 매우 짧은 응답
        )
        print(f"✅ 성공: {response.content}")
    except Exception as e:
        print(f"❌ 에러 발생: {type(e).__name__}")
        print(f"   메시지: {str(e)}")
        print("\n💡 가능한 해결 방법:")
        print("  - API 키 확인")
        print("  - 네트워크 연결 확인")
        print("  - Rate Limit 확인")
        print("  - 모델 이름 확인")


# 실행 (주석 해제)
# await error_handling_example()
print("💡 위의 주석을 해제하여 실행하세요.")

## 📸 시각화 예제

> 💡 **이미지 필요**: 아래는 시각화 예제입니다. 실제 사용 시 matplotlib이나 plotly를 사용하여 그래프를 생성하세요.

### 토큰 사용량 시각화

```python
import matplotlib.pyplot as plt

# 예제 데이터
models = ["gpt-4o", "claude-sonnet-4.5", "gemini-2.5-pro"]
tokens = [150, 180, 200]

plt.bar(models, tokens)
plt.title("토큰 사용량 비교")
plt.ylabel("토큰 수")
plt.xticks(rotation=45)
plt.show()
```

**필요한 이미지**: 
- 막대 그래프: 각 모델별 토큰 사용량 비교
- X축: 모델 이름
- Y축: 토큰 수


## 7. 다음 단계

Client 기본 사용법을 마스터했습니다! 다음 튜토리얼로 진행하세요:

### 📚 추천 학습 순서

1. **[03_rag_and_documents.ipynb](03_rag_and_documents.ipynb)** - RAG 및 문서 처리
   - 문서 로딩 및 처리
   - RAGChain으로 지식 기반 응답 생성

2. **[04_embeddings_vector_stores.ipynb](04_embeddings_vector_stores.ipynb)** - 임베딩 및 벡터 스토어
   - 텍스트 임베딩 생성
   - 벡터 스토어에 저장 및 검색

3. **[05_agent_and_tools.ipynb](05_agent_and_tools.ipynb)** - Agent 및 Tools
   - Tool을 사용하는 Agent 생성
   - 복잡한 작업 자동화

### 🔗 관련 문서

- [API Reference](../API_REFERENCE.md#client) - Client API 상세 문서
- [README.md](../../README.md) - 프로젝트 개요

---

**✅ Client 기본 사용법 완료! 다음 노트북으로 진행하세요.**
